In [2]:
!pip install pydub

In [29]:
from pydub import AudioSegment
import numpy as np
from scipy.fft import fft, ifft

input_filename=input("Please input the audio file:")
input_file=AudioSegment.from_file(input_filename)
#input_file=AudioSegment.from_file("test1.mp3")

noise_filename=input("Please input the noise file:")
noise_file=AudioSegment.from_file(noise_filename)
#noise_file=AudioSegment.from_file("noise.mp3")

input_array=np.array(input_file.get_array_of_samples())
noise_array=np.array(noise_file.get_array_of_samples())

for i in range(len(noise_array),len(input_array),1):
  noise_array.append(np.mean(np.abs(noise_array)))
noise_array=noise_array[:len(input_array)]
input_array=fft(input_array)
noise_array=fft(noise_array)

for i in range(0,len(input_array),1):
  if(np.abs(noise_array[i])>0.1*np.abs(input_array[i])):
    input_array[i]=0

output_array=ifft(input_array)


output=AudioSegment(output_array.astype(np.int16).tobytes(),
          frame_rate=input_file.frame_rate,
          sample_width=input_file.sample_width,
          channels=input_file.channels)
output.export("/content/output.mp3", format="mp3")

Please input the audio file:test1.mp3
Please input the noise file:noise.mp3


<ipython-input-29-a091e90fcdee>:29: ComplexWarning: Casting complex values to real discards the imaginary part
  output=AudioSegment(output_array.astype(np.int16).tobytes(),


<_io.BufferedRandom name='/content/output.mp3'>

In [13]:
# ChatGPT version

import numpy as np
import scipy.signal
import pydub
import soundfile as sf

def audio_to_numpy(audio):
    """将 pydub.AudioSegment 转换为 numpy 数组。"""
    samples = np.array(audio.get_array_of_samples())
    # 如果音频是立体声，选择其中一个通道
    if audio.channels == 2:
        samples = samples[::2]
    return samples.astype(np.float32)

def numpy_to_audio(samples, sample_rate):
    """将 numpy 数组转换为 pydub.AudioSegment。"""
    samples = samples.astype(np.int16)  # 转换为 16 位 PCM 格式
    return pydub.AudioSegment(
        samples.tobytes(),
        frame_rate=sample_rate,
        sample_width=2,
        channels=1
    )

def calculate_segment_max_amplitudes(signal, segment_length, hop_size):
    """计算每个小段的最大音量幅度。"""
    num_segments = (len(signal) - segment_length) // hop_size + 1
    max_amplitudes = np.zeros(num_segments)
    for i in range(num_segments):
        start = i * hop_size
        end = start + segment_length
        segment = signal[start:end]
        max_amplitudes[i] = np.max(np.abs(segment))
    return max_amplitudes

def denoise_audio(input_file, output_file, window_size=1024, hop_size=512, noise_fraction=0.005):
    # 读取音频文件
    audio = pydub.AudioSegment.from_file(input_file)
    sample_rate = audio.frame_rate

    # 将音频转换为 numpy 数组
    audio_signal = audio_to_numpy(audio)

    # 计算每个小段的最大音量
    max_amplitudes = calculate_segment_max_amplitudes(audio_signal, window_size, hop_size)
    print(f"每个小段的最大音量幅度: {max_amplitudes}")

    # 选择窗函数
    window = np.hanning(window_size)

    # 计算 STFT
    f, t, Zxx = scipy.signal.stft(audio_signal, fs=sample_rate, window=window, nperseg=window_size, noverlap=hop_size)


    # 根据每个小段的最大音量设置阈值
    Zxx_filtered = np.zeros_like(Zxx)
    for i in range(len(max_amplitudes)):
        segment_threshold = noise_fraction * max_amplitudes[i]
        Zxx_filtered[:, i] = np.where(np.abs(Zxx[:, i]) < segment_threshold, 0, Zxx[:, i])

    # 使用 inverse STFT 将频域信号转换回时域信号
    _, filtered_audio_signal = scipy.signal.istft(Zxx_filtered, fs=sample_rate, window=window, nperseg=window_size, noverlap=hop_size)

    # 将 numpy 数组转换回 pydub.AudioSegment
    filtered_audio = numpy_to_audio(filtered_audio_signal, sample_rate)

    # 保存处理后的音频文件
    filtered_audio.export(output_file, format="mp3")

# 调用函数进行音频降噪
denoise_audio('test2.mp3', 'file1.mp3')


每个小段的最大音量幅度: [   0.    0.    0. ... 1921. 1921. 1879.]
